In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sys
sys.path.append("../..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric
import logging

from src.utils import logging_utils

# logging_utils.configure(level=logging.DEBUG)

In [22]:
# stats = {}

# dataset = data.load_dataset()
# for relation in dataset:
#     stats[relation.name] = {
#         "name": relation.name,
#         "category": relation.properties.relation_type,
#         "num_samples": len(relation.samples)
#     }

# with open("stats/data.json", "w") as f:
#     json.dump(stats, f, indent=2)

In [23]:
# ############################################
# sweep_root = "../../results/sweep-24-trials"
# # sweep_root = "../../results/sweep-bare"
# model_name = "llama-13b" # gptj | gpt2-xl | llama-13b
# ############################################

# sweep_path = f"{sweep_root}/{model_name}"

In [24]:
# from src.utils.sweep_utils import read_sweep_results


# sweep_results = read_sweep_results(sweep_path, economy=False)
# list(sweep_results.keys())

In [25]:
# with open("stats/data.json", "r") as f:
#     stats = json.load(f)

# for relation in stats:
#     if relation not in sweep_results:
#         stats[relation][model_name] = f"--"
#         print(f"Skipping {relation} -- not found for {model_name}")
#         continue
#     trail_tests = np.array([
#         trial["n_test_samples"] 
#         for trial in sweep_results[relation]["trials"]
#     ])
#     # if len(trail_tests) < (3 if model_name != "llama-13b" else 2):
#     #     stats[relation][model_name] = f"--"
#     #     print(f"Skipping {relation} -- not enough trials for {model_name}")
#     #     continue
#     # else:
#     stats[relation][model_name] = f"{trail_tests.mean() :.2f} \pm {trail_tests.std() :.2f}"

# with open("stats/data.json", "w") as f:
#     json.dump(stats, f, indent=2)

## Table 4

In [26]:
with open("stats/data.json", "r") as f:
    stats = json.load(f)

In [29]:
def change_decimal(result):
    if result == "--":
        return result
    mean, std = result.split(" \pm ")
    mean = float(mean)
    std = float(std)
    return f"{mean:.1f} \pm {std:.1f}"

In [34]:
category = "bias"
category_stats = [
    stats[relation] 
    for relation in stats 
    if stats[relation]["category"] == category
]

category_stats = sorted(
    category_stats, key=lambda x: x["num_samples"], reverse=True
)
print(len(category_stats))
for relation in category_stats:
    relation_stats = relation
    relation_name = relation_stats["name"]
    gptj_count = change_decimal(relation_stats["gptj"])
    gpt2_count = change_decimal(relation_stats["gpt2-xl"])
    llama_count = change_decimal(relation_stats["llama-13b"])
    print(f"& {relation_name} & ${relation_stats['num_samples']}$ & ${gptj_count}$ & ${gpt2_count}$ & ${llama_count}$ \\\\ ")
print("\\hline")

7
& occupation age & $45$ & $25.7 \pm 2.6$ & $22.9 \pm 3.8$ & $32.8 \pm 2.6$ \\ 
& univ degree gender & $38$ & $--$ & $21.5 \pm 2.4$ & $24.2 \pm 2.4$ \\ 
& name birthplace & $31$ & $17.1 \pm 2.6$ & $18.0 \pm 1.4$ & $21.4 \pm 1.1$ \\ 
& name religion & $31$ & $17.0 \pm 2.3$ & $15.1 \pm 2.2$ & $19.8 \pm 1.5$ \\ 
& characteristic gender & $30$ & $15.9 \pm 2.7$ & $15.8 \pm 2.2$ & $19.7 \pm 1.2$ \\ 
& name gender & $19$ & $11.0 \pm 0.0$ & $10.7 \pm 0.6$ & $10.8 \pm 0.4$ \\ 
& occupation gender & $19$ & $9.6 \pm 0.8$ & $9.8 \pm 0.7$ & $10.8 \pm 0.4$ \\ 
\hline


In [9]:


change_decimal(relation_stats['gptj'])

'9.6 \\pm 0.8'

'9.60 \\pm 0.80'